In [1700]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [1701]:
data = pd.read_csv('../dataset/preprocessed_train_data')
data

,passenger_id,age,room_service,food_court,shopping_mall,spa,v_r_deck,name,transported,Europa,...,deck_D,deck_E,deck_F,deck_G,deck_T,deck_unknown,side_S,side_unknown,v_i_p_True,v_i_p_unknown
0,0001_01,39.0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0002_01,24.0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1,0,...,0,0,1,0,0,0,1,0,0,0
2,0003_01,58.0,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0,1,...,0,0,0,0,0,0,1,0,1,0
3,0003_02,33.0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0,1,...,0,0,0,0,0,0,1,0,0,0
4,0004_01,16.0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1,0,...,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,41.0,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,0,1,...,0,0,0,0,0,0,0,0,1,0
8689,9278_01,18.0,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,0,0,...,0,0,0,1,0,0,1,0,0,0
8690,9279_01,26.0,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,1,0,...,0,0,0,1,0,0,1,0,0,0
8691,9280_01,32.0,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,0,1,...,0,1,0,0,0,0,1,0,0,0


In [1702]:
data['last_name'] = data['name'].str.split().str[-1]
surname_counts = data['last_name'].value_counts()
data['has_family'] = data['last_name'].map(surname_counts > 1).fillna(0).astype(int)

In [1703]:
data

,passenger_id,age,room_service,food_court,shopping_mall,spa,v_r_deck,name,transported,Europa,...,deck_F,deck_G,deck_T,deck_unknown,side_S,side_unknown,v_i_p_True,v_i_p_unknown,last_name,has_family
0,0001_01,39.0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0,1,...,0,0,0,0,0,0,0,0,Ofracculy,0
1,0002_01,24.0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1,0,...,1,0,0,0,1,0,0,0,Vines,1
2,0003_01,58.0,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0,1,...,0,0,0,0,1,0,1,0,Susent,1
3,0003_02,33.0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0,1,...,0,0,0,0,1,0,0,0,Susent,1
4,0004_01,16.0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1,0,...,1,0,0,0,1,0,0,0,Santantines,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,41.0,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,0,1,...,0,0,0,0,0,0,1,0,Noxnuther,1
8689,9278_01,18.0,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,0,0,...,0,1,0,0,1,0,0,0,Mondalley,1
8690,9279_01,26.0,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,1,0,...,0,1,0,0,1,0,0,0,Connon,1
8691,9280_01,32.0,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,0,1,...,0,0,0,0,1,0,0,0,Hontichre,1


In [1704]:
passenger_id = data['passenger_id']
data = data.drop(columns='name')
data = data.drop(columns='passenger_id')
data = data.drop(columns='last_name')

In [1705]:
data

,age,room_service,food_court,shopping_mall,spa,v_r_deck,transported,Europa,Mars,unknown_planet,...,deck_E,deck_F,deck_G,deck_T,deck_unknown,side_S,side_unknown,v_i_p_True,v_i_p_unknown,has_family
0,39.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,24.0,109.0,9.0,25.0,549.0,44.0,1,0,0,0,...,0,1,0,0,0,1,0,0,0,1
2,58.0,43.0,3576.0,0.0,6715.0,49.0,0,1,0,0,...,0,0,0,0,0,1,0,1,0,1
3,33.0,0.0,1283.0,371.0,3329.0,193.0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,1
4,16.0,303.0,70.0,151.0,565.0,2.0,1,0,0,0,...,0,1,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,41.0,0.0,6819.0,0.0,1643.0,74.0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,1
8689,18.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,1
8690,26.0,0.0,0.0,1872.0,1.0,0.0,1,0,0,0,...,0,0,1,0,0,1,0,0,0,1
8691,32.0,0.0,1049.0,0.0,353.0,3235.0,0,1,0,0,...,1,0,0,0,0,1,0,0,0,1


In [1706]:
train_data = data.drop(columns='transported')
target = data['transported']

In [1707]:
X_train, X_test, y_train, y_test = train_test_split(train_data, target, test_size=0.2, stratify=target, random_state=42)

In [1708]:
scaler = MinMaxScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

In [1709]:
model_rf = RandomForestClassifier()
model_rf.fit(X_train, y_train)
model_rf.score(X_train, y_train)

0.999712395743457

In [1710]:
model_rf.score(X_test, y_test)

0.8108108108108109

In [1711]:
model_cat = CatBoostClassifier(
    iterations=2000,           
    learning_rate=0.02,        
    depth=8,                   
    l2_leaf_reg=6,             
    bagging_temperature=0.8,   
    random_strength=1.5,       
    border_count=128,         
    subsample=0.8,             
    colsample_bylevel=0.8,     
    random_seed=42,
    early_stopping_rounds=100,
)

model_cat.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    use_best_model=True,
    plot=False
)

print("CatBoost accuracy:", model_cat.score(X_test, y_test))

0:	learn: 0.6838380	test: 0.6838421	best: 0.6838421 (0)	total: 3.16ms	remaining: 6.32s
1:	learn: 0.6749981	test: 0.6750319	best: 0.6750319 (1)	total: 6.73ms	remaining: 6.72s
2:	learn: 0.6674105	test: 0.6673803	best: 0.6673803 (2)	total: 10.1ms	remaining: 6.72s
3:	learn: 0.6577345	test: 0.6577079	best: 0.6577079 (3)	total: 13.7ms	remaining: 6.81s
4:	learn: 0.6491773	test: 0.6491225	best: 0.6491225 (4)	total: 17.2ms	remaining: 6.87s
5:	learn: 0.6421730	test: 0.6420659	best: 0.6420659 (5)	total: 20.9ms	remaining: 6.94s
6:	learn: 0.6336488	test: 0.6336051	best: 0.6336051 (6)	total: 24.3ms	remaining: 6.91s
7:	learn: 0.6273192	test: 0.6272189	best: 0.6272189 (7)	total: 27.7ms	remaining: 6.89s
8:	learn: 0.6205824	test: 0.6201775	best: 0.6201775 (8)	total: 31.2ms	remaining: 6.91s
9:	learn: 0.6149908	test: 0.6148986	best: 0.6148986 (9)	total: 35.1ms	remaining: 6.99s
10:	learn: 0.6080261	test: 0.6079237	best: 0.6079237 (10)	total: 38.4ms	remaining: 6.94s
11:	learn: 0.6019315	test: 0.6020745	best

In [1712]:
test_data = pd.read_csv('../dataset/test.csv')
test_data

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter
4273,9269_01,Earth,False,NaN,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron
4274,9271_01,Mars,True,D/296/P,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore
4275,9273_01,Europa,False,D/297/P,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale


In [1713]:
test_data['last_name'] = test_data['Name'].str.split().str[-1]
surname_counts = test_data['last_name'].value_counts()
test_data['has_family'] = test_data['last_name'].map(surname_counts > 1).fillna(0).astype(int)
test_data = test_data.drop(columns='last_name')

In [1714]:
test_data.columns = test_data.columns.str.replace(r'(?<!^)(?=[A-Z])', '_', regex=True).str.lower()
test_data

,passenger_id,home_planet,cryo_sleep,cabin,destination,age,v_i_p,room_service,food_court,shopping_mall,spa,v_r_deck,name,has_family
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning,1
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers,0
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus,0
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter,0
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter,1
4273,9269_01,Earth,False,NaN,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron,1
4274,9271_01,Mars,True,D/296/P,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore,1
4275,9273_01,Europa,False,D/297/P,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale,1


In [1715]:
test_data['home_planet'] = test_data['home_planet'].fillna('unknown_planet')
data_home_planet = pd.get_dummies(test_data['home_planet'], drop_first=True).astype(int)
test_data = pd.concat([test_data.drop(columns='home_planet'), data_home_planet], axis=1)

test_data['cryo_sleep'] = test_data['cryo_sleep'].fillna('unknown')
data_cryo_sleep = pd.get_dummies(test_data['cryo_sleep'], drop_first=True, prefix='cryo_sleep').astype(int)
test_data = pd.concat([test_data.drop(columns='cryo_sleep'), data_cryo_sleep], axis=1)
test_data

,passenger_id,cabin,destination,age,v_i_p,room_service,food_court,shopping_mall,spa,v_r_deck,name,has_family,Europa,Mars,unknown_planet,cryo_sleep_True,cryo_sleep_unknown
0,0013_01,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning,1,0,0,0,1,0
1,0018_01,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers,0,0,0,0,0,0
2,0019_01,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus,0,1,0,0,1,0
3,0021_01,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter,0,1,0,0,0,0
4,0023_01,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,G/1496/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter,1,0,0,0,1,0
4273,9269_01,NaN,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron,1,0,0,0,0,0
4274,9271_01,D/296/P,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore,1,0,1,0,1,0
4275,9273_01,D/297/P,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale,1,1,0,0,0,0


In [1716]:
test_data[['deck', 'num', 'side']] = test_data['cabin'].str.split('/', expand=True)
test_data = test_data.drop(columns='cabin')

test_data['destination'] = test_data['destination'].fillna('unknown')
data_home_planet = pd.get_dummies(test_data['destination'], drop_first=True, prefix='planet').astype(int)
test_data = pd.concat([test_data.drop(columns='destination'), data_home_planet], axis=1)
test_data = test_data.rename(columns={
    'PSO J318.5-22': 'pso_j3128_5_22',
    'TRAPPIST-1e': 'trappist_1e'
})

test_data['deck'] = test_data['deck'].fillna('unknown')
data_deck = pd.get_dummies(test_data['deck'], drop_first=True, prefix='deck').astype(int)
test_data = pd.concat([test_data.drop(columns='deck'), data_deck], axis=1)

test_data['side'] = test_data['side'].fillna('unknown')
data_deck = pd.get_dummies(test_data['side'], drop_first=True, prefix='side').astype(int)
test_data = pd.concat([test_data.drop(columns='side'), data_deck], axis=1)
test_data['age'] = pd.to_numeric(test_data['age'], errors='coerce')
test_data['age'] = test_data['age'].fillna(test_data['age'].median())
test_data['v_i_p'] = test_data['v_i_p'].fillna('unknown')
data_deck = pd.get_dummies(test_data['v_i_p'], drop_first=True, prefix='v_i_p').astype(int)
test_data = pd.concat([test_data.drop(columns='v_i_p'), data_deck], axis=1)
test_data['room_service'] = pd.to_numeric(data['room_service'], errors='coerce')
test_data['room_service'] = test_data['room_service'].fillna(data['room_service'].median())

test_data['food_court'] = pd.to_numeric(test_data['food_court'], errors='coerce')
test_data['food_court'] = test_data['food_court'].fillna(test_data['food_court'].median())

test_data['shopping_mall'] = pd.to_numeric(test_data['shopping_mall'], errors='coerce')
test_data['shopping_mall'] = test_data['shopping_mall'].fillna(test_data['food_court'].median())

test_data['spa'] = pd.to_numeric(test_data['spa'], errors='coerce')
test_data['spa'] = test_data['spa'].fillna(test_data['spa'].median())

test_data['v_r_deck'] = pd.to_numeric(test_data['v_r_deck'], errors='coerce')
test_data['v_r_deck'] = test_data['v_r_deck'].fillna(test_data['v_r_deck'].median())

In [1717]:
test_data = test_data.drop(columns='name')
passenger_ids = test_data['passenger_id'].copy()
test_data = test_data.drop(columns='passenger_id')

In [1718]:
test_data

,age,room_service,food_court,shopping_mall,spa,v_r_deck,has_family,Europa,Mars,unknown_planet,...,deck_D,deck_E,deck_F,deck_G,deck_T,deck_unknown,side_S,side_unknown,v_i_p_True,v_i_p_unknown
0,27.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,...,0,0,0,1,0,0,1,0,0,0
1,19.0,109.0,9.0,0.0,2823.0,0.0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
2,31.0,43.0,0.0,0.0,0.0,0.0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,38.0,0.0,6652.0,0.0,181.0,585.0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
4,20.0,303.0,0.0,635.0,0.0,0.0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,34.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,...,0,0,0,1,0,0,1,0,0,0
4273,42.0,0.0,847.0,17.0,10.0,144.0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,0
4274,26.0,375.0,0.0,0.0,0.0,0.0,1,0,1,0,...,1,0,0,0,0,0,0,0,0,0
4275,26.0,0.0,2680.0,0.0,0.0,523.0,1,1,0,0,...,1,0,0,0,0,0,0,0,0,0


In [1719]:
test_data = test_data.drop(columns=['transported'], errors='ignore')
test_data = test_data.reindex(columns=X_train.columns, fill_value=0)

In [1720]:
X_test_final = pd.DataFrame(
    scaler.transform(test_data),
    columns=test_data.columns
)

In [1721]:
len(X_test_final)

4277

In [1722]:
predictions = model_cat.predict(X_test_final)
len(predictions)

4277

In [1723]:
submission = pd.DataFrame({
    'PassengerId': passenger_ids,
    'Transported': predictions.astype(bool)
})

submission.to_csv('submission.csv', index=False)
print("✅ Submission файлът е готов: submission.csv")

✅ Submission файлът е готов: submission.csv
